In [2]:
from sage.misc.prandom import randint
from sage.rings.all import ZZ, Zmod, PolynomialRing, GF
from sage.arith.all import next_prime, euler_phi, crt, xgcd
from sage.functions.log import log
from sage.functions.other import ceil
from sage.misc.functional import cyclotomic_polynomial, round
import numpy as np
from random import randint

# Parameter Generation

In [3]:
d = 3
m = 3
N = 2**12
n = euler_phi(N)
log_q = 20 # weird sagemath behavior when this gets too big ca 50
evaluation_degree = d*m
D = d*m
tiny_primes = [2,3,5,7,11,13,17,19,23,29]
tiny_prime_product = 1
for prime in tiny_primes:
    tiny_prime_product *= prime

In [4]:
var('Z')
g = cyclotomic_polynomial(N, var='Z')

In [5]:
primes = []
q = 1
i = 1
while log(q,2).n()<log_q:
    p = N*i+1
    if p.is_prime():
        q *= p
        primes.append(p)
    i += 1
primes

[12289, 40961]

In [6]:
log(q,2).n()

28.9070432192714

In [7]:
Z_q = Zmod(q)
Q = Z_q['Z'].quo(g,Z)

In [8]:
var('Y')
h = ZZ['Y'](1)
monomials = []
assert D < primes[0]
for i in range(D):
    mon = (ZZ[Y](Y) - ZZ[Y](i))
    h *= mon
    monomials.append(mon)
R = Q['Y'].quo(h,Y)

# BV11

In [ ]:
s = Q.random_element()

In [ ]:
def encrypt(s,mu):
    assert mu in Z_q
    assert s in Q
    a = R(Q.random_element())
    
    # TODO make the noise real
    e = randint(0,2)

    b = a*R(s)+d*e+mu
    return -a*R(Y)+b

In [ ]:
c1 = encrypt(s,1)
c2 = encrypt(s,2)
c = c1 * c2

In [ ]:
def decrypt(s,c):
    a = Q(c.lift()(Y=s)).lift()(Z=0).lift()
    return mod(a,d)

In [ ]:
decrypt(s,c)

# Database Interpolation

In [ ]:
P = PolynomialRing(GF(d),m,'x')
vars = P.gens()
vars

In [ ]:
a=np.array([[[1,1,0],[2,0,1],[2,0,1]],[[1,1,1],[0,0,2],[2,0,1]],[[1,1,1],[0,0,2],[2,0,1]]],dtype=np.int64)
b=[1,5,3]
def multivariate_interpolation(a, vars, m):
    # assertion: a is an m dimensional cube with length d
    if m==1:
        return GF(d)[vars[0]].lagrange_polynomial([(i,a[i])for i in range(d)])
    else:
        cs = np.zeros(tuple([d]*m), dtype=np.int64)
        for x in GF(d)**(m-1):
            g_x = GF(d)[vars[-1]].lagrange_polynomial([(i,a[tuple(x)][i])for i in range(d)])
            c_x = g_x.list()
            for i in range(len(c_x)):
                cs[tuple(x)][i] = c_x[i]

        f_is = []
        f = 0
        for i in GF(d):
            f_i = multivariate_interpolation(cs[..., i], vars[:-1], m-1)
            f_is.append(f_i)
            f += f_i * vars[-1]**i
        return f
f = multivariate_interpolation(a,vars,3)
print(f)
f(0,1,0)

# Eval

In [ ]:
def normal_eval(Q,f,cs):
    f_prime =f.change_ring(Q)
    return f_prime(cs)

In [ ]:
normal_eval(R,f,[c1,c2,c1])

In [ ]:
def split_R_q(R_q_element):
    return [Zmod(prime)[Y,Z](R_q_element) for prime in primes]

def combine_R_q(R_p_elements):
    return Z_q[Y,Z](crt([ZZ[Y,Z](c) for c in R_p_elements],primes))

In [ ]:
def split_R_p(R_p_element,p):
    return [Zmod(p)[Y,Z].quo(mon)(R_p_element.lift()) for mon in monomials]

def combine_R_p(Q_p_elements):
    Z_p = Zmod(p)
    bezout_coefficients = []
    for i in range(len(monomials)):
        unit_i_vector = [1 if i == j else 0 for j in range(len(monomials))]
        bezout_coefficients.append(Z_p[Y,Z](crt(unit_i_vector,[Z_p[Y](mon) for mon in monomials])))
    #TODO above this should move to precomputation
    result = Z_p[Y,Z](0)
    for i in range(len(Q_p_elements)):
        result += Q_p_elements[i].lift()*bezout_coefficients[i]
    return R(result)

In [ ]:
def split_Q_p(Q_p_element,p):
    Z_p = Zmod(p)
    w_n = Z_p.multiplicative_generator()**((p-1)//N)
    assert w_n.multiplicative_order() == N
    frac_ps = [Z_p[Z](Z)-Z_p[Z](w_n**i) for i in Zmod(N).list_of_elements_of_multiplicative_group()]
    #TODO above this should move to precomputation
    return [Z_p[Z].quo(frac_p,Z)(Q_p_element) for frac_p in frac_ps]

def combine_Q_p(Z_p_elements,p):
    Z_p = Zmod(p)
    w_n = Z_p.multiplicative_generator()**((p-1)//N)
    assert w_n.multiplicative_order() == N
    frac_ps = [Z_p[Z](Z)-Z_p[Z](w_n**i) for i in Zmod(N).list_of_elements_of_multiplicative_group()]
    #TODO above this should move to precomputation
    #TODO move bezout coefficient computation to precomputation
    return Q(crt([c.lift() for c in Z_p_elements],frac_ps))


In [ ]:
def split_Z(Z_element,upper_bound):
    assert Z_element <= upper_bound
    assert upper_bound < tiny_prime_product
    i = 0
    while used_primes_product < upper_bound:
        used_primes_product *= tiny_primes[i]
        i += 1
    #TODO above this should move to precomputation
    return [Zmod(prime)(Z_element) for prime in tiny_primes[:i]]

def combine_Z(tiny_Z_elements,upper_bound):
    assert tiny_Z_elements <= upper_bound
    assert upper_bound < tiny_prime_product
    used_primes_product = 1
    i = 0
    while used_primes_product < upper_bound:
        used_primes_product *= tiny_primes[i]
        i += 1
    #TODO above this should move to precomputation
    #TODO move bezout coefficient computation to precomputation
    return crt([ZZ(c) for c in tiny_Z_elements],tiny_primes[:i])

In [ ]:
def precompute_polynomial(f,input_upper_bound):
    return np.from_function(f,tuple([input_upper_bound]*m))